- Sequential - Framework for NN
- Dense - Fully connected layer with weights in each neuron
- Input - Input layer
- Flatten - Convert n-D data to 1-D
- Activation - Activation layers. They just apply a function to the input.
- Mnist - Dataset with handwritten digits.

In [99]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pandas as pd
import numpy as np
import tensorflow as tf

In [61]:
data = pd.read_csv("all_japanese_stocks.csv")

In [62]:
data.head()

,Unnamed: 0,Date,High,Low,Open,Close,Volume,Adj Close
0,1301,2001-01-01,1400.0,1400.0,1400.0,1400.0,0.0,1003.504517
1,1301,2001-01-02,1400.0,1400.0,1400.0,1400.0,0.0,1003.504517
2,1301,2001-01-03,1400.0,1400.0,1400.0,1400.0,0.0,1003.504517
3,1301,2001-01-04,1420.0,1370.0,1420.0,1390.0,19300.0,996.336853
4,1301,2001-01-05,1400.0,1330.0,1390.0,1330.0,19700.0,953.329529


In [87]:
data['Date'] = pd.to_datetime(data['Date'])
data['Date'] = (data['Date'] - data['Date'].min()).dt.days

In [88]:
x = data[["High", "Low", "Open", "Close", "Adj Close"]]

In [89]:
input_dim = len(x.columns)
model = Sequential([
        # Encoder
        Dense(64, activation='relu', input_shape=(input_dim,)),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),  # Bottleneck layer
        
        # Decoder
        Dense(32, activation='relu'),
        Dense(64, activation='relu'),
        Dense(input_dim)
])

In [90]:
model.compile(
    loss="mse",
    optimizer="adam",
    metrics=["accuracy"]
)

In [91]:
x_in = x.head(1000)

In [92]:
x_tensor = tf.convert_to_tensor(x_in)

In [93]:
model.fit(x_tensor, x_tensor, epochs=100, batch_size=32, verbose=1)

Epoch 1/100
32/32 [==============================] - 1s 2ms/step - loss: 1409726.2500 - accuracy: 0.4600
Epoch 2/100
32/32 [==============================] - 0s 2ms/step - loss: 52733.0156 - accuracy: 0.0000e+00
Epoch 3/100
32/32 [==============================] - 0s 2ms/step - loss: 2589.4966 - accuracy: 0.7050 
Epoch 4/100
32/32 [==============================] - 0s 2ms/step - loss: 523.9653 - accuracy: 1.0000
Epoch 5/100
32/32 [==============================] - 0s 2ms/step - loss: 436.0072 - accuracy: 1.0000
Epoch 6/100
32/32 [==============================] - 0s 1ms/step - loss: 432.8030 - accuracy: 1.0000
Epoch 7/100
32/32 [==============================] - 0s 1ms/step - loss: 432.7235 - accuracy: 1.0000
Epoch 8/100
32/32 [==============================] - 0s 1ms/step - loss: 434.6611 - accuracy: 1.0000
Epoch 9/100
32/32 [==============================] - 0s 1ms/step - loss: 437.2708 - accuracy: 1.0000
Epoch 10/100
32/32 [==============================] - 0s 2ms/step - loss: 434.3

In [95]:
test_out = model.predict(x_tensor)
test_out.shape

32/32 [==============================] - 0s 1ms/step


(1000, 5)

In [104]:
loss_fn = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)

losses = loss_fn(x_tensor, test_out)

normal = np.percentile(losses, 99)

In [105]:
anomaly = losses[losses > normal]
len(anomaly)

10